In [2]:
import warnings
warnings.simplefilter("ignore")
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import pandas as pd
sys.path.append('/home/ubuntu/Mask_RCNN/samples/coco') 
import coco
%matplotlib inline
import re
import itertools 

# Analysis of both training (below are # out) and testing datasets

Using TensorFlow backend.


# Getting all Files Ready

In [7]:
# Read xml annotation files vs. output object counts -- # objects per frame

def getGroundTruthObjectCount(path):    
    df_annotation = pd.read_csv(path)
    df_annotation = df_annotation.drop(columns='Unnamed: 0')
    df_annotation_count = df_annotation.pivot_table(index = 'imageID', columns='annot_objs', aggfunc= lambda x:len(x)).fillna(0)
    #df_annotation_count = df_annotation_count.drop(columns='w')
    return df_annotation_count


def getPredObjectCount(path):
    ObjCount_files = os.listdir(path)

    a, b, c, d = [], [], [], []
    for file in ObjCount_files:
        if file.endswith('_0005.h5.txt'):
            each_file = pd.read_csv(path + file)
            each_file['ImageID'] = file.replace('_mask_rcnn_clown_cfg_0005.h5.txt','')
            each_file['epoch'] = '0005'
            a.append(each_file)
            df_a = pd.concat(a)
            df_a = df_a.drop(columns={'Unnamed: 0', 'image'})

        elif file.endswith('_0015.h5.txt'):
            each_file = pd.read_csv(path + file)
            each_file['ImageID'] = file.replace('_mask_rcnn_clown_cfg_0015.h5.txt','')
            each_file['epoch'] = '0015'
            b.append(each_file)
            df_b = pd.concat(b)
            df_b = df_b.drop(columns={'Unnamed: 0', 'image'})


        elif file.endswith('_0025.h5.txt'):
            each_file = pd.read_csv(path + file)
            each_file['ImageID'] = file.replace('_mask_rcnn_clown_cfg_0025.h5.txt','')
            each_file['epoch'] = '0025'
            c.append(each_file)
            df_c = pd.concat(c)
            df_c = df_c.drop(columns={'Unnamed: 0', 'image'})


        elif file.endswith('_0038.h5.txt'):
            each_file = pd.read_csv(path + file)
            each_file['ImageID'] = file.replace('_mask_rcnn_clown_cfg_0038.h5.txt','')
            each_file['epoch'] = '0038'
            d.append(each_file)
            df_d = pd.concat(d)
            df_d = df_d.drop(columns={'Unnamed: 0', 'image'})

    df_OutObj = pd.concat([df_a, df_b, df_c, df_d])
    return df_OutObj



if __name__ == "__main__":
    
    #df_groundTruthTraining = getGroundTruthObjectCount('/home/ubuntu/Mask_RCNN/data/trainingAnnotation.txt')
    #df_groundTruthTraining.reset_index(inplace=True)
    #df_groundTruthTraining['files'] = df_groundTruthTraining['imageID']
    #df_groundTruthTraining = df_groundTruthTraining.drop(columns={'w', "imageID"})
    #df_groundTruthTraining['files'] = df_groundTruthTraining['files'].str.replace('.xml','')
    #df_groundTruthTraining.to_csv('/home/ubuntu/Mask_RCNN/data/reform_trainingAnnotation.txt')
    
    df_groundTruthVal = getGroundTruthObjectCount('/home/ubuntu/Mask_RCNN/data/testingAnnotation.txt')
    df_groundTruthVal.reset_index(inplace=True)
    df_groundTruthVal['files'] = df_groundTruthVal['imageID']
    df_groundTruthVal = df_groundTruthVal.drop(columns={"imageID"})
    df_groundTruthVal['files'] = df_groundTruthVal['files'].str.replace('.xml','')
    df_groundTruthVal.to_csv('/home/ubuntu/Mask_RCNN/data/reform_testingAnnotation.txt')
    
    #df_PredTraining = getPredObjectCount('/home/ubuntu/Mask_RCNN/data/TrainingResults/outputObjCount/')
    #df_PredTraining.to_csv('/home/ubuntu/Mask_RCNN/data/reform_TrainingoutputObjCount.txt')
    
    df_PredVal = getPredObjectCount('/home/ubuntu/Mask_RCNN/data/outputObjCount/')
    df_PredVal.to_csv('/home/ubuntu/Mask_RCNN/data/reform_TestingoutputObjCount.txt')

In [196]:
# togethers - groundTruth vs. prediction (trainig set)- skip if there is no trainingset data 
df_index_training = pd.read_csv('/home/ubuntu/Mask_RCNN/data/TrainfilevsImageID.txt')
df_index_training['files'] = df_index_training['files'].str.replace("dataset.",'')
df_index_training = df_index.rename(columns= {'Unnamed: 0':'imageID'})
df_index_training['imageID'] = df_index_training['imageID'].replace('.xml', "")

truth = pd.merge(df_groundTruthTraining, df_index_training, on = ['files'], how ='outer')
truth = truth.rename(columns={'clown':'true_clown', 'color':'true_color', 'nface':'true_nface' })

df_PredTraining = df_PredTraining.rename(columns = {'ImageID':'imageID'})

all_file = []
for e in df_PredTraining.epoch.unique():
    dfsub = df_PredTraining[(df_PredTraining.epoch == e)]
    dfsub['imageID'] = dfsub['imageID'].astype(int)
    submerge = pd.merge(truth, dfsub, on=['imageID'], how='outer')
    all_file.append(submerge)
    
df1 = all_file[0]
df2 = all_file[1]
df3 = all_file[2]
df4 = all_file[3]
training_truth_pred_objCount = pd.concat([df1, df2, df3, df4])
training_truth_pred_objCount.to_csv('/home/ubuntu/Mask_RCNN/data/Training_truth_pred_objCount.txt')

In [44]:
# togethers - groundTruth vs. prediction (validation set)
df_index_testing = pd.read_csv('/home/ubuntu/Mask_RCNN/data/TestingfilevsImageID.txt')
df_index_testing['files'] = df_index_testing['files'].str.replace("dataset.",'')
df_index_testing = df_index_testing.rename(columns= {'Unnamed: 0':'imageID'})
df_index_testing['imageID'] = df_index_testing['imageID'].replace('.xml', "")


truth_val = pd.merge(df_groundTruthVal, df_index_testing, on = ['files'], how ='outer')
truth_val = truth_val.rename(columns={'clown':'val_clown', 'color':'val_color', 'nface':'val_nface' })

df_PredVal = df_PredVal.rename(columns = {'ImageID':'imageID'})

all_file_val = []
for e in df_PredVal.epoch.unique():
    dfsub_val = df_PredVal[(df_PredVal.epoch == e)]
    dfsub_val['imageID'] = dfsub_val['imageID'].astype(int)
    submerge_val = pd.merge(truth_val, dfsub_val, on=['imageID'], how='outer')
    all_file_val.append(submerge_val)
    
df1_val = all_file_val[0]
df2_val = all_file_val[1]
df3_val = all_file_val[2]
df4_val = all_file_val[3]
testing_truth_pred_objCount = pd.concat([df1_val, df2_val, df3_val, df4_val])
testing_truth_pred_objCount.to_csv('/home/ubuntu/Mask_RCNN/data/Testing_truth_pred_objCount.txt')

# Analysis Starts 

In [45]:
def generateNNPPTNNPforEpoch(data_file):

    NN = data_file[(data_file.true_clown == 0.0) & (
    data_file['#clown'] == 0.0)]

    PP = data_file[(data_file.true_clown != 0.0) & (
    data_file['#clown'] != 0.0)] 

    NP = data_file[(data_file.true_clown == 0.0) & (
    data_file['#clown'] != 0.0)] 

    PN = data_file[(data_file.true_clown != 0.0) & (
    data_file['#clown'] == 0.0)] 

    nn = NN.pivot_table(index='epoch', aggfunc = lambda x:len(x))
    nn.reset_index(inplace=True)
    try:
        nn['NN'] = nn['#clown'] #use any
        nn = nn.drop(columns={'#clown', '#nface', 'files', 'imageID', 'true_clown','true_color', 'true_nface'})
    except:
        pass

    pp = PP.pivot_table(index='epoch', aggfunc = lambda x:len(x))
    pp.reset_index(inplace=True)
    try:
        pp['PP'] = pp['#clown'] #use any
        pp = pp.drop(columns={'#clown', '#nface', 'files', 'imageID', 'true_clown','true_color', 'true_nface'})
    except:
        pass

    np = NP.pivot_table(index='epoch', aggfunc = lambda x:len(x))
    np.reset_index(inplace=True)
    try:
        np['NP'] = np['#clown'] #use any
        np = np.drop(columns={'#clown', '#nface', 'files', 'imageID', 'true_clown','true_color', 'true_nface'})
    except:
        pass

    pn = PN.pivot_table(index='epoch', aggfunc = lambda x:len(x))
    pn.reset_index(inplace=True)
    try:
        pn['PN'] = pn['#clown'] #use any
        pn = pn.drop(columns={'#clown', '#nface', 'files', 'imageID', 'true_clown','true_color', 'true_nface'})
    except:
        pass

    df_all = pd.concat([nn, pp, np, pn], axis = 1).drop_duplicates()
    df_all = df_all.loc[:,~df_all.columns.duplicated()]
    
    return df_all


if __name__ == "__main__":
    
    # cut out google images 
    #training_truth_pred_objCount = training_truth_pred_objCount.sort_values(by=['files']).iloc[0:2464]
    #df_training_PPNNPNNP_tabel = generateNNPPTNNPforEpoch(training_truth_pred_objCount)
    #df_training_PPNNPNNP_tabel.to_csv('/home/ubuntu/Mask_RCNN/data/training_forConfusionTablePlot.txt')
    
   
    testing_truth_pred_objCount = testing_truth_pred_objCount.rename(columns={'val_clown':'true_clown',
                                           'val_color':'true_color',
                                           'val_nface':'true_nface'}).fillna(0)
    df_testing_PPNNPNNP_tabel = generateNNPPTNNPforEpoch(testing_truth_pred_objCount).fillna(0)
    df_testing_PPNNPNNP_tabel.to_csv('/home/ubuntu/Mask_RCNN/data/testing_forConfusionTablePlot.txt')